## Miscellaneous

In [1]:
import pandas as pd

from custom_lib import df_info, display_time_interval, compare_schema, compare_schema_table

In [2]:
ems = pd.read_csv('EMS.csv')
fire = pd.read_csv('FIRE.csv')
fire_stations = pd.read_csv('Firehouse.csv')

In [3]:
df_info(ems, rows=2)
df_info(fire, rows=2)
df_info(fire_stations, rows=2)

DataFrame: ems
Shape: (4781652, 31)
+----+-------------------+------------------------+---------------------+-------------------------------+-------------------+-----------------------------+-----------------------------+----------------------------------+--------------------------------+-----------------------------+---------------------------+----------------------------------+--------------------------------+---------------------------------+--------------------------+-------------------------------+---------------------------+------------------+-----------------------------+-----------+--------------------------+-----------+------------------+-----------------------+---------------------+---------------------------+-------------------------+--------------------+---------------------------+---------------------+----------------------+
|    |   CAD_INCIDENT_ID | INCIDENT_DATETIME      | INITIAL_CALL_TYPE   |   INITIAL_SEVERITY_LEVEL_CODE | FINAL_CALL_TYPE   |   FINAL_SEVERITY_LEVEL_C

## Basic analysis

In [6]:
display_time_interval(ems)
print()
display_time_interval(fire)

start    : 2022-01-01 01:00:20
end      : 2025-01-01 00:59:47
duration : 1095 days 23:59:27

start    : 2022-01-01 01:00:53
end      : 2025-01-01 00:58:59
duration : 1095 days 23:58:06


In [ ]:
res = compare_schema(ems, fire)
schema_df = compare_schema_table(ems, fire)
df_info(schema_df, rows=50)

common_df = schema_df[schema_df["in_ems"] & schema_df["in_fire"]]
only_fire_df = schema_df[~schema_df["in_ems"] & schema_df["in_fire"]]
only_ems_df = schema_df[schema_df["in_ems"] & ~schema_df["in_fire"]]
dtype_mismatch_df = common_df[common_df["ems_dtype"] != common_df["fire_dtype"]]

DataFrame: schema_df
Shape: (44, 5)
+----+--------------------------------+----------+-----------+-------------+--------------+
|    | column                         | in_ems   | in_fire   | ems_dtype   | fire_dtype   |
|----+--------------------------------+----------+-----------+-------------+--------------|
|  7 | CITYCOUNCILDISTRICT            | True     | True      | float64     | float64      |
|  8 | COMMUNITYDISTRICT              | True     | True      | float64     | float64      |
|  9 | COMMUNITYSCHOOLDISTRICT        | True     | True      | float64     | float64      |
| 10 | CONGRESSIONALDISTRICT          | True     | True      | float64     | float64      |
| 11 | DISPATCH_RESPONSE_SECONDS_QY   | True     | True      | object      | object       |
| 15 | FIRST_ACTIVATION_DATETIME      | True     | True      | object      | object       |
| 16 | FIRST_ASSIGNMENT_DATETIME      | True     | True      | object      | object       |
| 18 | FIRST_ON_SCENE_DATETIME        | True

In [11]:
fire_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   FacilityName        219 non-null    object 
 1   FacilityAddress     219 non-null    object 
 2   Borough             219 non-null    object 
 3   Postcode            219 non-null    int64  
 4   Latitude            219 non-null    float64
 5   Longitude           219 non-null    float64
 6   Community Board     219 non-null    int64  
 7   Community Council   219 non-null    int64  
 8   Census Tract        219 non-null    object 
 9   BIN                 219 non-null    object 
 10  BBL                 219 non-null    object 
 11  NTA                 219 non-null    object 
dtypes: float64(2), int64(3), object(7)
memory usage: 20.7+ KB


### Grain 
Each dataframe has a strict one-row-per-incident grain. Incident identifiers are unique, non-null, and stable. This is exactly what you want for fact tables. No fix needed, I think :)

In [ ]:
def check_grain(df, key):
    return {
        "rows": len(df),
        "unique_key": df[key].nunique(dropna=True),
        "null_key": int(df[key].isna().sum()),
        "duplicate_key_rows": int(df.duplicated(key).sum())
    }

check_grain(ems, "CAD_INCIDENT_ID"), check_grain(fire, "STARFIRE_INCIDENT_ID")

({'rows': 4781652,
  'unique_key': 4781652,
  'null_key': 0,
  'duplicate_key_rows': 0},
 {'rows': 2020279,
  'unique_key': 2020279,
  'null_key': 0,
  'duplicate_key_rows': 0})

### Time coverage and ordering
Both datasets cover the same time window (2022–2025). Datetimes parse cleanly, with only a handful of illogical orderings (e.g. on-scene before incident time). This is negligible at scale. 

We could flag those rows in staging and exclude them from duration KPIs, but keep them in the facts. Negligeable honestly

In [13]:
def check_datetime(df, col="INCIDENT_DATETIME"):
    dt = pd.to_datetime(df[col], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")
    return {
        "null_raw": int(df[col].isna().sum()),
        "unparsed": int(dt.isna().sum()),
        "min": dt.min(),
        "max": dt.max(),
    }

check_datetime(ems), check_datetime(fire)

({'null_raw': 0,
  'unparsed': 0,
  'min': Timestamp('2022-01-01 01:00:20'),
  'max': Timestamp('2025-01-01 00:59:47')},
 {'null_raw': 0,
  'unparsed': 0,
  'min': Timestamp('2022-01-01 01:00:53'),
  'max': Timestamp('2025-01-01 00:58:59')})

sanity on event ordering (should be non-decreasing when present):

In [14]:
def check_order(df, a, b, fmt="%m/%d/%Y %I:%M:%S %p"):
    A = pd.to_datetime(df[a], format=fmt, errors="coerce")
    B = pd.to_datetime(df[b], format=fmt, errors="coerce")
    bad = (A.notna() & B.notna() & (B < A)).sum()
    return {"bad_pairs": int(bad), "checked_pairs": int((A.notna() & B.notna()).sum())}

check_order(fire, "INCIDENT_DATETIME", "FIRST_ON_SCENE_DATETIME")


{'bad_pairs': 5, 'checked_pairs': 1485549}

### Measures quality (response times)
Response time fields contain non-numeric values, sentinel caps (`999`), and negative values (FIRE). Left as-is, this will break averages and percentiles.
Huuuuh during staging we can cast to numeric and apply rules:

* values `< 0` → `NULL`
* values `>= 999` → `NULL`
* keep `NULL` (no imputation)
  Store cleaned fields in facts; keep raw fields in staging for traceability.


In [15]:
def check_numeric(df, cols):
    out = {}
    for c in cols:
        s = pd.to_numeric(df[c], errors="coerce")
        out[c] = {
            "null_raw": int(df[c].isna().sum()),
            "non_numeric": int(s.isna().sum() - df[c].isna().sum()),
            "min": s.min(),
            "max": s.max(),
        }
    return out

shared_measures = [
    "DISPATCH_RESPONSE_SECONDS_QY",
    "INCIDENT_RESPONSE_SECONDS_QY",
    "INCIDENT_TRAVEL_TM_SECONDS_QY",
]
check_numeric(ems, shared_measures), check_numeric(fire, shared_measures)

({'DISPATCH_RESPONSE_SECONDS_QY': {'null_raw': 0,
   'non_numeric': 175021,
   'min': np.float64(0.0),
   'max': np.float64(999.0)},
  'INCIDENT_RESPONSE_SECONDS_QY': {'null_raw': 226223,
   'non_numeric': 698579,
   'min': np.float64(0.0),
   'max': np.float64(999.0)},
  'INCIDENT_TRAVEL_TM_SECONDS_QY': {'null_raw': 225383,
   'non_numeric': 441193,
   'min': np.float64(0.0),
   'max': np.float64(999.0)}},
 {'DISPATCH_RESPONSE_SECONDS_QY': {'null_raw': 921,
   'non_numeric': 5908,
   'min': np.float64(-20.0),
   'max': np.float64(999.0)},
  'INCIDENT_RESPONSE_SECONDS_QY': {'null_raw': 534730,
   'non_numeric': 27003,
   'min': np.float64(9.0),
   'max': np.float64(999.0)},
  'INCIDENT_TRAVEL_TM_SECONDS_QY': {'null_raw': 534758,
   'non_numeric': 18350,
   'min': np.float64(-964.0),
   'max': np.float64(999.0)}})

### Categorical attributes
EMS call types and FIRE classification groups are clean, low-cardinality, and domain-stable. They are good candidates for dimensions and will aggregate well in BI. No fix needed beyond basic text normalization (trim, upper).

In [16]:
def top_values(df, col, n=20):
    return df[col].value_counts(dropna=False).head(n)

top_values(ems, "FINAL_CALL_TYPE"), top_values(fire, "INCIDENT_CLASSIFICATION_GROUP")


(FINAL_CALL_TYPE
 SICK      582326
 INJURY    547901
 UNKNOW    385154
 DRUG      300262
 UNC       262306
 CARD      238118
 CARDBR    226395
 EDPC      219198
 ABDPN     193195
 INJMAJ    177253
 EDP       151311
 DIFFBR    146064
 MVAINJ    125571
 ALTMEN     91156
 STATEP     81272
 ARREST     79979
 EDPM       78938
 PEDSTR     77639
 CDBRFC     75035
 OTHER      59670
 Name: count, dtype: int64,
 INCIDENT_CLASSIFICATION_GROUP
 Medical Emergencies       1111195
 NonMedical Emergencies     723857
 Structural Fires            72545
 NonMedical MFAs             65319
 NonStructural Fires         39202
 Medical MFAs                 8161
 Name: count, dtype: int64)

### Firehouse linkage
The firehouse table is clean, but there is no reliable key linking incidents to stations. Borough naming mismatches exist, and there is no station or unit id in the FIRE facts.

We can try those:
* normalize borough labels across all tables
* treat `Dim_Firehouse` as independent in v1
* plan a phase-2 bridge using geospatial nearest-station logic or a future unit/station identifier if available

Net: you are ready to build the model after applying the fixes for points 3 and 5.


In [ ]:
firehouse_boroughs = set(fire_stations["Borough"].astype(str).str.upper().str.strip())
fire_boroughs = set(fire["INCIDENT_BOROUGH"].astype(str).str.upper().str.strip())
sorted(fire_boroughs - firehouse_boroughs)[:20]

['RICHMOND / STATEN ISLAND']

## Database creation

In [4]:
from pathlib import Path

from galaxy_tools import (
    DT_COLS_COMMON, DT_COLS_EMS_EXTRA, LOC_COLS, MEASURE_COLS_SHARED,
    make_staging, build_dim_time, build_dim_location, build_small_dims, build_dim_firehouse,
    build_fact_ems, build_fact_fire, export_parquet
)

OUT_DIR = Path("powerbi_parquet")
OUT_DIR.mkdir(exist_ok=True)

creates your staging tables, ``ems_c`` and `fire_c`. You’re not changing the raw data, you’re making cleaned working copies. Datetimes become real timestamps, borough names are standardized into a shared column, and response time fields are cleaned so invalid values become **null**. Everything downstream relies on these tables.

In [5]:
ems_c, fire_c = make_staging(ems, fire)

builds the two core shared dimensions. 
- `Dim_Time` is one row per day and lets you do trends, year-over-year, monthly views, weekday effects. 
- `Dim_Location` is one row per unique geography combination and lets you slice EMS and FIRE the same way. These two dimensions are reused everywhere and are the backbone of the model.

In [6]:
dim_time = build_dim_time(
    ems_c, fire_c,
    datetime_cols=list(set(DT_COLS_COMMON + DT_COLS_EMS_EXTRA))
)

dim_location = build_dim_location(ems_c, fire_c, loc_cols=LOC_COLS)

df_info(dim_time, rows=5)
df_info(dim_location, rows=5)

DataFrame: dim_time
Shape: (1097, 7)
+----+---------------------+------------+--------+---------+-------+-----------+--------+
|    | date                |   date_key |   year |   month |   day |   weekday |   week |
|----+---------------------+------------+--------+---------+-------+-----------+--------|
|  0 | 2022-01-01 00:00:00 |   20220101 |   2022 |       1 |     1 |         5 |     52 |
|  1 | 2022-01-02 00:00:00 |   20220102 |   2022 |       1 |     2 |         6 |     52 |
|  2 | 2022-01-03 00:00:00 |   20220103 |   2022 |       1 |     3 |         0 |      1 |
|  3 | 2022-01-04 00:00:00 |   20220104 |   2022 |       1 |     4 |         1 |      1 |
|  4 | 2022-01-05 00:00:00 |   20220105 |   2022 |       1 |     5 |         2 |      1 |
+----+---------------------+------------+--------+---------+-------+-----------+--------+
DataFrame: dim_location
Shape: (2056, 8)
+----+----------------+----------------+-----------+------------------+-----------------------+-----------------

builds the small domain dimensions. These are things like EMS call type, severity, disposition, and FIRE classification and alarm info. You move text categories out of the fact tables so your facts stay lean and your slicers in Power BI stay clean and stable. This is what makes analysis readable instead of messy. Our favourite AI brought them to me, I don't actually know if they will be relevant, I dunno, it's a lot of micro connections so I think we won't use all of them, we'll see.

In [7]:
dims_other = build_small_dims(ems_c, fire_c)
list(dims_other.keys())

['dim_ems_initial_call_type',
 'dim_ems_final_call_type',
 'dim_ems_initial_severity',
 'dim_ems_final_severity',
 'dim_ems_disposition',
 'dim_fire_class_group',
 'dim_fire_class',
 'dim_fire_alarm_source',
 'dim_fire_alarm_level']

builds `Dim_Firehouse`. This table describes stations and where they are. For now it mostly gives context and supports borough-level analysis. You’re setting yourself up for later, when you can link incidents to stations more precisely.

In [8]:
dim_firehouse = build_dim_firehouse(fire_stations)
df_info(dim_firehouse, rows=5)

DataFrame: dim_firehouse
Shape: (219, 14)
+----+-----------------+---------------------------------------------------------+--------------------+-----------+------------+------------+-------------+-------------------+----------------------+----------------+-----------+---------------+----------------------------------------+----------------+
|    |   firehouse_key | FacilityName                                            | FacilityAddress    | Borough   |   Postcode |   Latitude |   Longitude |   Community Board |   Community Council  | Census Tract   | BIN       | BBL           | NTA                                    | Borough_Norm   |
|----+-----------------+---------------------------------------------------------+--------------------+-----------+------------+------------+-------------+-------------------+----------------------+----------------+-----------+---------------+----------------------------------------+----------------|
|  0 |               1 | Engine 4/Ladder 15         

- `Fact_Incident_EMS` is one row per EMS incident with response times, flags, and foreign keys to time, location, and EMS dimensions. 
- `Fact_Incident_FIRE` is one row per fire incident with response times, unit counts, alarm info, and the same shared keys. 

You keep them separate because they represent different operational realities, even though they share dimensions.

In [9]:
fact_ems = build_fact_ems(ems_c, dim_location, dims_other, measures_shared=MEASURE_COLS_SHARED)
fact_fire = build_fact_fire(fire_c, dim_location, dims_other, measures_shared=MEASURE_COLS_SHARED)

df_info(fact_ems, rows=5)
df_info(fact_fire, rows=5)

DataFrame: fact_ems
Shape: (4781652, 17)
+----+-------------------+---------------------+---------------------+----------------+----------------------------------------+----------------------------------------+-----------------------------------------+-----------------------------+---------------------------+----------------------------+--------------------------+-----------------------+------------------+--------------------+---------------------------+---------------------+----------------------+
|    |   ems_incident_id | incident_datetime   |   incident_date_key |   location_key |   dispatch_response_seconds_qy_seconds |   incident_response_seconds_qy_seconds |   incident_travel_tm_seconds_qy_seconds |   ems_initial_call_type_key |   ems_final_call_type_key |   ems_initial_severity_key |   ems_final_severity_key |   ems_disposition_key |   held_indicator |   reopen_indicator |   special_event_indicator |   standby_indicator |   transfer_indicator |
|----+-------------------+-------

This writes every table as Parquet into one folder. Power BI imports Parquet fast, keeps types better than CSV, and won’t choke on millions of rows as easily.

In [10]:
export_parquet(dim_time, OUT_DIR / "dim_time.parquet")
export_parquet(dim_location, OUT_DIR / "dim_location.parquet")
export_parquet(dim_firehouse, OUT_DIR / "dim_firehouse.parquet")

for name, df in dims_other.items():
    export_parquet(df, OUT_DIR / f"{name}.parquet")

export_parquet(fact_ems, OUT_DIR / "fact_incident_ems.parquet")
export_parquet(fact_fire, OUT_DIR / "fact_incident_fire.parquet")

sorted([p.name for p in OUT_DIR.glob("*.parquet")])


['dim_ems_disposition.parquet',
 'dim_ems_final_call_type.parquet',
 'dim_ems_final_severity.parquet',
 'dim_ems_initial_call_type.parquet',
 'dim_ems_initial_severity.parquet',
 'dim_fire_alarm_level.parquet',
 'dim_fire_alarm_source.parquet',
 'dim_fire_class.parquet',
 'dim_fire_class_group.parquet',
 'dim_firehouse.parquet',
 'dim_location.parquet',
 'dim_time.parquet',
 'fact_incident_ems.parquet',
 'fact_incident_fire.parquet']